In [1]:
##paso 1##
import pandas as pd
import numpy as np

dataset_test = pd.read_csv(
    "../files/input/test_data.csv.zip",
    index_col=False,
    compression="zip",
)
dataset_train = pd.read_csv(
    "../files/input/train_data.csv.zip",
    index_col=False,
    compression="zip",
)

# - Renombre la columna "default payment next month" a "default".
dataset_test.rename(columns={"default payment next month": "default"}, inplace=True)
dataset_train.rename(columns={"default payment next month": "default"}, inplace=True)

# - Remueva la columna "ID".
dataset_test.drop(columns=["ID"], inplace=True)
dataset_train.drop(columns=["ID"], inplace=True)

# - Elimine los registros con informacion no disponible.
dataset_test.dropna(inplace=True)
dataset_train.dropna(inplace=True)

# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others".
dataset_test.loc[dataset_test["EDUCATION"] > 4, "EDUCATION"] = 4
dataset_train.loc[dataset_train["EDUCATION"] > 4, "EDUCATION"] = 4

dataset_train

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,310000,1,3,1,32,0,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,10000,2,3,1,49,-1,-1,-2,-1,2,...,1690,1138,930,0,0,2828,0,182,0,1
2,50000,1,2,1,28,-1,-1,-1,0,-1,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,80000,2,3,1,52,2,2,3,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,270000,1,1,2,34,1,2,0,0,2,...,22448,15490,17343,0,4000,2000,0,2000,2000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,140000,2,2,1,27,2,-1,-1,-1,0,...,1580,804,728,752,800,1580,0,700,700,0
20996,130000,1,2,2,41,0,0,0,0,0,...,123107,42897,39378,4442,5200,5012,2500,5000,2000,0
20997,50000,1,3,2,23,0,0,0,0,0,...,28967,29829,30046,1973,1426,1001,1432,1062,997,0
20998,90000,2,3,2,25,0,0,0,0,0,...,5613,10113,10113,3000,3000,0,4500,0,3440,0


In [2]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
y_train=dataset_train.copy()
y_train=y_train.pop("default")

x_train = dataset_train.copy()
x_train.pop("default")

y_test=dataset_test.copy()
y_test=y_test.pop("default")

x_test = dataset_test.copy()
x_test.pop("default")

0       1
1       0
2       0
3       0
4       0
       ..
8995    0
8996    0
8997    0
8998    1
8999    1
Name: default, Length: 9000, dtype: int64

In [3]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
# - Estandariza la matriz de entrada.
# - Selecciona las K columnas mas relevantes de la matrix de entrada.
# - Ajusta una maquina de vectores de soporte (svm).

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC

#'variables_categoricas' es una lista de nombres de columnas categóricas
variables_categoricas = ["SEX", "EDUCATION", "MARRIAGE"] 
numeric_columns = [col for col in x_train.columns if col not in variables_categoricas]

# Preprocesador para las columnas categóricas
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# Preprocesador combinado para los datos
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, variables_categoricas),                 
        ('num', StandardScaler(), numeric_columns),  
    ],
)

# Crear el pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),                
    ("pca", PCA()),                                              
    ("feature_selection", SelectKBest(score_func=f_classif)),  
    ("classifier", SVC(random_state=97))  
])

print("Pipeline creado con éxito.")


Pipeline creado con éxito.


In [4]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score

# Definir el rango de hiperparámetros para optimizar
param_grid = {
    "pca__n_components": [None],  
    "feature_selection__k": [17],                     
    "classifier__gamma": [0.1], 
}

# Crear el evaluador de precisión balanceada
scorer = make_scorer(balanced_accuracy_score)

# Configurar GridSearchCV
best_model = GridSearchCV(
    estimator=pipeline,          
    param_grid=param_grid,       
    scoring=scorer,               
    cv=10,                       
    n_jobs=-1,                    
    verbose=1                    
)

# Ajustar el GridSearchCV al conjunto de datos de entrenamiento
best_model.fit(x_train, y_train)

# Mostrar los mejores parámetros y el rendimiento
print("Mejores hiperparámetros encontrados:")
print(best_model.best_params_)
print(f"Mejor puntuación de validación (balanced accuracy): {best_model.best_score_:.4f}")

# Actualizar el pipeline con los mejores parámetros encontrados
best_pipeline = best_model.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Mejores hiperparámetros encontrados:
{'classifier__gamma': 0.1, 'feature_selection__k': 17, 'pca__n_components': None}
Mejor puntuación de validación (balanced accuracy): 0.6495


In [5]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

import os
import gzip
import pickle

# Guardar el objeto GridSearchCV completo
ruta_modelo = '../files/models/model.pkl.gz'
os.makedirs(os.path.dirname(ruta_modelo), exist_ok=True)
with gzip.open(ruta_modelo, 'wb') as f:
     pickle.dump(best_model, f)

In [6]:
# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

from sklearn.metrics import precision_score, recall_score, f1_score
import json

# Calcular las metricas
metrics = { 'type': "metrics",
    'dataset': 'train',
    'precision': precision_score(y_train, best_pipeline.predict(x_train)),
    'balanced_accuracy': balanced_accuracy_score(y_train, best_pipeline.predict(x_train)),
    'recall': recall_score(y_train, best_pipeline.predict(x_train)),
    'f1_score': f1_score(y_train, best_pipeline.predict(x_train)),
}

metrics_test = { 'type': "metrics",
    'dataset': 'test',
    'precision': precision_score(y_test, best_pipeline.predict(x_test)),
    'balanced_accuracy': balanced_accuracy_score(y_test, best_pipeline.predict(x_test)),
    'recall': recall_score(y_test, best_pipeline.predict(x_test)),
    'f1_score': f1_score(y_test, best_pipeline.predict(x_test)),}


# Guardar las métricas en formato JSON
ruta= "../files/output"
ruta_json= os.path.join(ruta, "metrics.json")

# Crear el directorio
os.makedirs(ruta, exist_ok=True)

with open(ruta_json, 'w') as f:
    json.dump(metrics, f, ensure_ascii=False) 
    f.write('\n')
    json.dump(metrics_test, f, ensure_ascii=False)  
    f.write('\n')

In [7]:
# Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#

from sklearn.metrics import confusion_matrix

# Calcular las matrices de confusión
cm_train = confusion_matrix(y_train, best_pipeline.predict(x_train))
cm_test = confusion_matrix(y_test, best_pipeline.predict(x_test))

cm_train_dict = {
    'type': 'cm_matrix',
    'dataset': 'train',
    'true_0': {'predicted_0': int(cm_train[0, 0]), 'predicted_1': int(cm_train[0, 1])},
    'true_1': {'predicted_0': int(cm_train[1, 0]), 'predicted_1': int(cm_train[1, 1])}
}

cm_test_dict = {
    'type': 'cm_matrix',
    'dataset': 'test',
    'true_0': {'predicted_0': int(cm_test[0, 0]), 'predicted_1': int(cm_test[0, 1])},
    'true_1': {'predicted_0': int(cm_test[1, 0]), 'predicted_1': int(cm_test[1, 1])}
}


#guardar en json
with open(ruta_json, 'a', encoding='utf-8') as f:
    json.dump(cm_train_dict, f, ensure_ascii=False) 
    f.write('\n')
    json.dump(cm_test_dict, f, ensure_ascii=False)  
    f.write('\n')